# Nifti Read Example

The purpose of this notebook is to illustrate reading Nifti files and iterating over patches of the volumes loaded from them.

References
https://github.com/Project-MONAI/tutorials/blob/master/modules/nifti_read_example.ipynb


## Setup environment

In [1]:
#!python -c "import monai" || pip install -q "monai-weekly[nibabel]"

## Setup imports

In [2]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import glob
import os
import shutil
import tempfile

import nibabel as nib
import numpy as np
import torch

from monai.config import print_config
from monai.data import (
    ArrayDataset, GridPatchDataset, create_test_image_3d, PatchIter)
from monai.transforms import (
    AddChannel,
    Compose,
    LoadImage,
    RandSpatialCrop,
    ScaleIntensity,
    EnsureType,
)
from monai.utils import first

print_config()

MONAI version: 0.8.0
Numpy version: 1.21.2
Pytorch version: 1.9.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 714d00dffe6653e21260160666c4c201ab66511b

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 3.2.2
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: 8.4.0
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: 4.3.0
TorchVision version: 0.10.1
tqdm version: 4.62.3
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: NOT INSTALLED or UNKNOWN VERSION.
pandas version: 1.4.1
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable. (e.g. export MONAI_DATA_DIRECTORY=$HOME/datasets/monai/ #set MONAI_DATA_DIRECTORY environment variable)
This allows you to save results and reuse downloads.  
If not specified a temporary directory will be used.

In [4]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/home/mx19/datasets/monai/


Create a number of test Nifti files:

In [5]:
for i in range(5):
    im, seg = create_test_image_3d(128, 128, 128)

    n = nib.Nifti1Image(im, np.eye(4))
    nib.save(n, os.path.join(root_dir, f"im{i}.nii.gz"))

    n = nib.Nifti1Image(seg, np.eye(4))
    nib.save(n, os.path.join(root_dir, f"seg{i}.nii.gz"))

Create a data loader which yields uniform random patches from loaded Nifti files:

In [6]:
images = sorted(glob.glob(os.path.join(root_dir, "im*.nii.gz")))
segs = sorted(glob.glob(os.path.join(root_dir, "seg*.nii.gz")))

imtrans = Compose(
    [
        LoadImage(image_only=True),
        ScaleIntensity(),
        AddChannel(),
        RandSpatialCrop((64, 64, 64), random_size=False),
        EnsureType(),
    ]
)

segtrans = Compose(
    [
        LoadImage(image_only=True),
        AddChannel(),
        RandSpatialCrop((64, 64, 64), random_size=False),
        EnsureType(),
    ]
)

ds = ArrayDataset(images, imtrans, segs, segtrans)

loader = torch.utils.data.DataLoader(
    ds, batch_size=10, num_workers=2, pin_memory=torch.cuda.is_available()
)
im, seg = first(loader)
print(im.shape, seg.shape)

torch.Size([5, 1, 64, 64, 64]) torch.Size([5, 1, 64, 64, 64])


Alternatively create a data loader which yields patches in regular grid order from loaded images:

In [9]:
imtrans = Compose([LoadImage(image_only=True),
                   ScaleIntensity(), AddChannel(), EnsureType()])

segtrans = Compose([LoadImage(image_only=True), AddChannel(), EnsureType()])

ds = ArrayDataset(images, imtrans, segs, segtrans)
patch_iter = PatchIter(patch_size=(64, 64, 64), start_pos=(0, 0, 0))


def img_seg_iter(x):
    return (zip(patch_iter(x[0]), patch_iter(x[1])),)


ds = GridPatchDataset(ds, img_seg_iter, with_coordinates=False)

loader = torch.utils.data.DataLoader(
    ds, batch_size=10, num_workers=2, pin_memory=torch.cuda.is_available()
)
im, seg = first(loader)
print("image shapes:", im[0].shape, seg[0].shape)
print("coordinates shapes:", im[1].shape, seg[1].shape)

image shapes: torch.Size([3, 1, 64, 64, 64]) torch.Size([3, 1, 64, 64, 64])
coordinates shapes: torch.Size([3, 4, 2]) torch.Size([3, 4, 2])


## Cleanup data directory

Remove directory if a temporary was used.

In [12]:
if directory is None:
    shutil.rmtree(root_dir)